In [4]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

sub_id = "b746917e-ceb7-4ae0-81e6-3ccd893cb0de"
rg = "dpv2"
workspace = "dpv2-wks"

ml_client = MLClient(DefaultAzureCredential(),  sub_id, rg, workspace)

In [5]:
from azure.ai.ml.entities import AmlCompute

cluster = ml_client.compute.get("cpu-cluster")

print(cluster)

AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'cpu-cluster', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/b746917e-ceb7-4ae0-81e6-3ccd893cb0de/resourceGroups/dpv2/providers/Microsoft.MachineLearningServices/workspaces/dpv2-wks/computes/cpu-cluster', 'Resource__source_path': None, 'base_path': 'd:\\code\\azureml-examples\\sdk\\jobs\\single-step\\lightgbm\\iris', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000001C629B4F5B0>, 'resource_id': None, 'location': 'eastus', 'size': 'STANDARD_DS3_V2', 'min_instances': 0, 'max_instances': 2, 'idle_time_before_scale_down': 120.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': None, 'tier': 'dedicated'})


In [19]:
from azure.ai.ml import command, Input

job = command(
    code = "./src",
    command = "python main.py --iris-csv ${{inputs.iris}} --learning-rate ${{inputs.lr}} --boosting ${{inputs.boosting}}",
    environment="AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu@latest",
    inputs={
        "iris": Input(type="uri_file", path="https://azuremlexamples.blob.core.windows.net/datasets/iris.csv",),
        "lr": 0.9,
        "boosting": "gbdt",
    },
    compute="cpu-cluster",     
)

In [20]:
ret_job = ml_client.jobs.create_or_update(job)
ret_job.services["Studio"].endpoint

'https://ml.azure.com/runs/shy_gas_4k8h824pbv?wsid=/subscriptions/b746917e-ceb7-4ae0-81e6-3ccd893cb0de/resourcegroups/dpv2/workspaces/dpv2-wks&tid=9c37dc6f-e051-496f-832c-e4fcf894c611'

In [10]:
print(ret_job)

$schema: '{}'
type: command
inputs:
  iris:
    mode: ro_mount
    type: uri_file
    path: azureml:https://azuremlexamples.blob.core.windows.net/datasets/iris.csv
  lr: '0.9'
  boosting: gbdt
outputs:
  default:
    mode: rw_mount
    type: uri_folder
    path: azureml://datastores/workspaceartifactstore/ExperimentRun/dcid.frosty_whale_x1zc7l5323
command: python.py main.py --iris-csv ${{inputs.iris}} --learning-rate ${{inputs.lr}}
  --boosting ${{inputs.boosting}}
environment: azureml:AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu:46
code: azureml:/subscriptions/b746917e-ceb7-4ae0-81e6-3ccd893cb0de/resourceGroups/dpv2/providers/Microsoft.MachineLearningServices/workspaces/dpv2-wks/codes/b5200222-af6b-438d-a5d6-b035ff8256d7/versions/1
resources:
  instance_count: 1
  properties: {}
compute: azureml:cpu-cluster
environment_variables: {}
component:
  name: frosty_whale_x1zc7l5323
  display_name: frosty_whale_x1zc7l5323
  type: command
  inputs:
    iris:
      type: uri_file
    lr:
      typ

In [23]:
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy
sweep_job = job(
    lr = Uniform(min_value=0.01, max_value=0.9),
    boosting=Choice(values=["gbdt","dart"]),
)

In [24]:
sweep_job_def = sweep_job.sweep(
    sampling_algorithm="random",
    primary_metric="test-multi_logloss",
    goal="Minimize"
)

sweep_job_def.set_limits(max_total_trials=20, max_concurrent_trials=10, timeout=7200)

In [25]:
ret_job = ml_client.create_or_update(sweep_job_def)
ret_job.services["Studio"].endpoint

'https://ml.azure.com/runs/sincere_tray_tmrxvkcy7b?wsid=/subscriptions/b746917e-ceb7-4ae0-81e6-3ccd893cb0de/resourcegroups/dpv2/workspaces/dpv2-wks&tid=9c37dc6f-e051-496f-832c-e4fcf894c611'

In [26]:
# define component
iris_componet = command(
    code = "./src",
    command = "python main.py --iris-csv ${{inputs.iris}} --learning-rate ${{inputs.lr}} --boosting ${{inputs.boosting}}",
    environment="AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu@latest",
    inputs={
        "iris": Input(type="uri_file", path="https://azuremlexamples.blob.core.windows.net/datasets/iris.csv",),
        "lr": 0.9,
        "boosting": "gbdt",
    },
    display_name = "iris lightgbm example",     
    description="Train a lightgbm model on the iris dataset"
)



In [ ]:
# run sweep using this component
# this time we will provide in input for iris_csv since no default was provide at compnent definition
command_component_job_for_sweep = iris_component(
    learning_rate=Uniform(min_value=0.01, max_value=0.9),
    boosting=Choice(values=["gbdt", "dart"]),
    iris_csv=Input(
        type="uri_file",
        path="wasbs://datasets@azuremlexamples.blob.core.windows.net/iris.csv",
    ),
)

# this is the same as above
cmd_component_sweep_job = command_component_job_for_sweep.sweep(
    compute="cpu-cluster",
    sampling_algorithm="random",
    primary_metric="test-multi_logloss",
    goal="Minimize",
)

cmd_component_sweep_job.display_name = "lightgbm-iris-sweep-example"
cmd_component_sweep_job.experiment_name = "lightgbm-iris-sweep-example"
cmd_component_sweep_job.description = (
    "Run a hyperparameter sweep job for LightGBM on Iris dataset."
)

# define the limits for this sweep
cmd_component_sweep_job.set_limits(
    max_total_trials=20, max_concurrent_trials=10, timeout=7200
)

# set early stopping on this one
cmd_component_sweep_job.early_termination = MedianStoppingPolicy(
    delay_evaluation=5, evaluation_interval=2
)